# Import Libraries

In [38]:
#Standard
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import math as m
import pandas as pd
import scipy.stats as stats
from scipy.stats import iqr, kurtosis, skew
from tqdm import tnrange, tqdm_notebook
from statannot import add_stat_annotation

#import pillow (PIL) to allow for image cropping
import PIL 
from PIL import Image, ImageChops
from io import BytesIO

#image simplification and priming
#Convolution libraries
from scipy import signal
from skimage.measure import label, regionprops
from sklearn.preprocessing import Binarizer
#from sklearn.preprocessing import Binarizer
from scipy import ndimage


#Skimage used for direct detection ellipse
from skimage import io
from skimage import data, color, img_as_ubyte
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter
from skimage.transform import rescale, resize, downscale_local_mean

#Skimage used for direct detection circles
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter

#OpenCV for circle detection
import cv2

# Common Functions

In [39]:
#This allows for the cutting of black space from each uCT image
def trim2(im,padding,offset):
    #selecting the outermost pixels 
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, offset)
    bbox = diff.getbbox()
    #adding small boarder to each image 
    bbox = np.array(bbox).reshape(2,2)
    bbox[0] -= padding
    bbox[1] += padding
    bbox = bbox.flatten()
    bbox = tuple(bbox)
    if bbox:
        return bbox,padding

#obscure convolutes 2D arrays with a (x,y) sized screen screen and then binarizes them
def obscure(image_array,x,y,invert):
    screen = np.ones((x,y), dtype=int) 
    image_array = signal.convolve2d(image_array,screen, mode='same') #,mode='same')
    #convert image into binary
    #image_array = np.where(image_array > 127.5, 1, 0)
    if invert == 'yes':
        image_array = np.where(image_array > 127.5, 0, 1)
    elif invert == 'no':
        image_array = np.where(image_array > 127.5, 1, 0)
    return image_array

#This allows for the additon of a padding to numpy array - useful for adding boarders to images
#found: https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 10)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
    return vector
    
def reject_outliers(data, m = 2):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/int(mdev) if mdev else 0
    return data[s<m]

##Create file if does not exist
def checkdir(dir):
    #First check if directory exists
    if os.path.isdir(dir) == False:
        os.makedirs(dir)
    else:
        pass

# Load Image Data

In [40]:
#Initialisation of data read
#current location
location = os.getcwd()

#where is data located    
loc = '/Volumes/RISTO_EXHDD/uCT'
# loc = '/Users/ristomartin/OneDrive/Dropbox/UniStuff/DPhil/Experimental/python_analysis/uCT/hollow_fibre'
# loc = '/Volumes/Ristos_SSD/uCT'

#What is the name of the data set?
data_set = 'S4_50PPM_8HRS_5PX'#'S4_10PPM_03_5PX_1_Rec'

data_loc = loc+'/'+data_set+'/'+data_set+'_Rec2'

# data_loc = loc+'/'+'test'+'/'
#location for saved data
save_loc = '/Users/ristomartin/OneDrive/Dropbox/UniStuff/DPhil/Experimental/python_analysis/uCT/flat_sheet/output/'
# save_loc = r'C:\Users\Alex Witt\Documents\Python_Analysis\Outputs'

#Check that the save location exists
checkdir(save_loc)

#what to name to save files
savename = data_set

#initilisation of constants
#Set conversion of px to um
pxum = 5

# Processing Images

In [41]:
#Define Image processing script as function to be entered into multiprocessing
def fibrefeature(dat_loc,filename,pxum,fibre_pad,fibre_scale,img_no,rotate,debug,debug_print,save_pic):
    #check whether to save picture or not
    save_pic = save_pic
    #Intially open full image
    im = Image.open(dat_loc+'/'+filename)
    #check if file needs converting
    if im.mode == 'I;16':
        #specifying its sampling mode
        im.mode = 'I'
        #convert the mode into 'L' (8-bit pixels, black and white) and save as temporary file
        im = im.point(lambda i:i*(1./256)).convert('L')
        #open temporaray file
        #im = Image.open('temp.jpeg')
    elif im.mode == 'RGB' or im.mode == 'RGBA':
        im = im.convert('L')
    else:
        pass

    
    #Once image is opened make copies of unedited image and array to use later on
    #make copy of original unadultorated image
    im_orig = im.copy()
    #as well as an array of the unadultorated image
    im_orig_array = np.array(im_orig)
    #A = (A * B)
    #im_orig_array = im_orig_array*(255.0/im_orig_array.max())
    
    im_orig_array = im_orig_array.astype("uint8")
    
    #create plot of convoluted and binarised image
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(im_orig_array, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'raw_image.png', dpi=300)
            
    ##Make selection of top RHS to find average pixel value to subtract
    bg_x1 = round((0.90*im_orig_array.shape[1]))
    bg_x2 = round((0.99*im_orig_array.shape[1]))
    bg_y1 = round((0.90*im_orig_array.shape[0]))
    bg_y2 = round((0.99*im_orig_array.shape[0]))
    
    # print(bg_x1)
    # print(bg_x2)
    # print(bg_y1)
    # print(bg_y2)
    
    bg_select = im_orig_array[bg_y1:bg_y2,bg_x1:bg_x2]
    # print(bg_select)
    
    # bg_select_df = pd.DataFrame(bg_select)
    # bg_select_df.to_csv(save_loc+savename+'bg_select.csv')

#     ret4,bg_select = cv2.threshold(im_orig_array,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    #create plot selected image background
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(bg_select, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'_bg_select.png', dpi=300)
    
    # height, width = bg_select.shape
    # bg_med = []
    # for i in range(0, width):
    #     #temp = (bg_select[i,:] > 0.1) * bg_select[i,:]
    #     temp = bg_select[i,:][bg_select[i,:]!=0]
    #     bg_med.extend(temp)
    
    bg_med = bg_select.flatten()

    #print(bg_med)
    bg_select_med = np.median(bg_med)+2*np.std(bg_med)  
    bg_select_mean = np.mean(bg_med)+2*np.std(bg_med)
    
    im_orig_array_c = im_orig_array.copy()
    
    #ret,bg_select = cv2.threshold(im_orig_array,bg_select_mean,im_orig_array.max(),cv2.THRESH_BINARY)
    im_orig_array_c[im_orig_array_c < bg_select_mean] = 0
    
    #create plot selected image background
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(im_orig_array_c, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'_thresh_bg_select.png', dpi=300)
            
    im = Image.fromarray(im_orig_array_c)
    
    
    im_debug = np.array(im)
    #create plot selected image background
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(im_debug, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'im_debug.png', dpi=300)

    #trim image to just pixels of interest using trim as defined above
    fibre_box,fpadding = trim2(im,(fibre_pad*2),trim_offset)
    im = im.crop(fibre_box)
    #convert trimmed image into numpy array
    nim =  np.array(im)
    #make copy of trimmed image to be used later on if needed
    nim_copy = nim.copy()
    
    #create plot of convoluted and binarised image
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(nim_copy, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'cropped_image.png', dpi=300)
    
    #Reduce image size
    nim = rescale(nim, fibre_scale, anti_aliasing=False)
    nim = np.uint8(nim * 255)
            
############################################################################################################################################################ 
                                                ### OUTER WALL DETECTION  ###
############################################################################################################################################################
    
    ##-- priming image for further analysis with obscure as defined above
    x = 7
    y = 7
    #Initially applying GayssuanBlur to minimise noise in image
    nim = cv2.GaussianBlur(nim,(x,y),0)
    #apply OTSU's binarisation method to strip away as much noise as possible and convert image into binary
    ret,fibre_thresh = cv2.threshold(nim,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    x = 2
    y = 2
    #1. Mark all pixels such that there are at least 6 pixels in their 7x7 neighborhood this will become defined as obscure function
    nim = signal.convolve2d(fibre_thresh,np.ones((x,y), dtype=int), mode='same')
    

    #create plot of convoluted and binarised image
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(nim, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'convolve2d_image.png', dpi=300)

    #need to find overall orientation of image and rotate to make horizontal
    #to initially find orientation to trim image to leave behind only narrow region of interest this must be in both the x and y axis
    #to ensure this is only done for first image make if gate to prevent multi runs
    coords = 0 
    # img_no = 0
    if img_no == 0:
        #get location of all detected true pixels
        coords = np.column_stack(np.where(nim == 255))
        # print(coords)
        #consider the spread of data in each direction, as considering flat membranes expect smaller spread in direction of normal to the face of the membrane
        #find the IQR in x_axis
        iqr_x = iqr(coords[:,0])
        #find the median in the x-axis
        median_x = np.median(coords[:,0])
        # #consider data in only y axis
        # data = coords[:,0]
        #find the IQR in y_axis
        iqr_y = iqr(coords[:,1])
        #print(iqr_y)
        if iqr_x < iqr_y:
            #make note to rotate
            rotate = 1

    if rotate == 1:    
        #rotate the image
        #th1 = ndimage.rotate(th1, 90,reshape=False )
        nim = nim.swapaxes(-2,-1)[...,::-1]
        # print(nim)
        #get location of all detected true pixels
        coords = np.column_stack(np.where(nim == 255))
        #reconsier the median of the x axis as that previously of the y axis due to rotation
        median_x = np.median(coords[:,0])
        #reconsider IQR as well
        iqr_x = iqr(coords[:,0])
    
        
        
    #create plot of convoluted and binarised image
    if debug == True:
        fig, ax = plt.subplots()
        ax.imshow(nim, 'gray')
        if save_pic == True:
            ax.figure.savefig(save_loc+filename+'rotated_convolve2d_image.png', dpi=300)

    #Covert list of coordinates into pandas dataframe
    coords = pd.DataFrame(coords)
    #get unique y-axis points at which pixels are detected
    unique_vals = pd.unique(coords[0].values)
    
    #Make list to hold all of the membrane thicknesses
    thicknesses = []
    
    #Itterating through each of the unique y-axis points
    for i in unique_vals:
        #isolate only the data associated with y-axis
        temp = coords.loc[coords[0] == i][1]
        #Find the median and IQR of each line
        Q1 = temp.quantile(0.25)
        Q3 = temp.quantile(0.75)
        IQR = Q3 - Q1
        median = temp.quantile(0.5)
        #convert temp from series to list
        temp = temp.tolist()
        #remove any values from temp which are more than 2 IQR from median
        temp = [x if abs(x-median)<(2*IQR) else median for x in temp]
        #find how thick membrane is at each y axis point
        if max(temp)-min(temp) == 0:
            pass
        else:
            thicknesses.append(max(temp)-min(temp))
    # print(thicknesses)
    
    #Convert list of thicknesses to an array so that stats may be determined
    thicknesses = np.array(thicknesses)
    #Calculate the stats associated with membrane thicknesses
    thick_mean = np.mean(thicknesses)*(1/fibre_scale)*pxum
    thick_med = (np.median(thicknesses)/fibre_scale)*pxum
    q75, q25 = (np.percentile(thicknesses, [75 ,25])/fibre_scale)*pxum
    thick_IQR = q75 - q25



    
    
############################################################################################################################################################ 
                                        ### Save out  ###
############################################################################################################################################################
    return (thick_mean,thick_med,thick_IQR,rotate)





In [42]:
test = True
show_all = False
debug_print = False

columns = ['filename','thick_IQR','thick_mean','thick_med']

cfp = pd.DataFrame(columns = columns)

if test == True:
    #initilisation of constants - Set conversion of px to um
    pxum = 5
    wire_diameter = 300
    fibre_pad = 50
    fibre_scale = 0.25
    trim_offset = -80
    
    #Generate list of file in data location
    files = [x for x in os.listdir(data_loc) if x.endswith(('.tif','.jpg','.png','.bmp'))==True and x.startswith('._')==False]
    
    #Make counter for file number
    img_no = -1
    rotate = 0
    #Itterating through files in data location
    for filename in tqdm_notebook(files):     
        #proceed image count
        img_no = img_no+1 
        #acertain fibre properties as defined above
        if show_all == True:
            print(filename)
            flatmem_properties = fibrefeature(data_loc,filename,pxum,fibre_pad,fibre_scale,img_no,rotate,True,True,True)
        else:
            # print(filename)
            flatmem_properties = fibrefeature(data_loc,filename,pxum,fibre_pad,fibre_scale,img_no,rotate,False,False,False)
            
        if flatmem_properties is None:
            pass
        else:
            # print(flatmem_properties)
            rotate = flatmem_properties[3]
            cfp = cfp.append({'filename':filename,'thick_mean':flatmem_properties[0],'thick_med':flatmem_properties[1],'thick_IQR':flatmem_properties[2]}, ignore_index=True)
    print(cfp.head())
    cfp.to_csv(save_loc+savename+'.csv')

        


/var/folders/dw/7hyg5yn15331npl8_ss6cl_40000gn/T/ipykernel_2035/1193030535.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for filename in tqdm_notebook(files):


  0%|          | 0/1334 [00:00<?, ?it/s]

                         filename  thick_IQR  thick_mean  thick_med
0  S4_50PPM_8HRS_5PX__rec0002.tif       40.0  150.851064      160.0
1  S4_50PPM_8HRS_5PX__rec0003.tif       20.0  147.741935      160.0
2  S4_50PPM_8HRS_5PX__rec0004.tif       40.0  154.594595      160.0
3  S4_50PPM_8HRS_5PX__rec0005.tif       40.0  155.957447      160.0
4  S4_50PPM_8HRS_5PX__rec0006.tif       40.0  149.723757      160.0


# Adding metadata

In [43]:
#Initially open processed data csv file
processed_flat = pd.read_csv(save_loc + 'processed_flat.csv',index_col = 0)

#For each of the rows in the processed data csv file match the corresponding sample file to associated metadata
for file, row in processed_flat.iterrows():
    #processed_flat.iloc[:,0]:
    #print(file)
    processed_flat.loc[file, 'pyridine_conc'] = sample_key.loc[sample_key['uCT_filename'] == file, 'pyridine_conc'].iloc[0]
    processed_flat.loc[file, 'rotation_speed'] = sample_key.loc[sample_key['uCT_filename'] == file, 'rotation_speed'].iloc[0]
    processed_flat.loc[file, 'solution_name'] = sample_key.loc[sample_key['uCT_filename'] == file, 'solution_name'].iloc[0]
    processed_flat.loc[file, 'time_spun'] = sample_key.loc[sample_key['uCT_filename'] == file, 'time_spun'].iloc[0]
    voltage = sample_key.loc[sample_key['uCT_filename'] == file, 'voltage'].iloc[0]
    min_voltage = sample_key.loc[sample_key['uCT_filename'] == file, 'min_voltage'].iloc[0]
    max_voltage = sample_key.loc[sample_key['uCT_filename'] == file, 'max_voltage'].iloc[0]
    processed_flat.loc[file, 'Voltage Range'] = (((voltage-min_voltage)/(max_voltage-min_voltage))*100).round(0)
#Having collated all the meta data check correctly recorded
print(processed_flat.head())
#save pandas data frame as CSV
processed_flat.to_csv(save_loc + 'processed_flat.csv')
#cdf.to_csv(save_loc+'MicroCT/porosity_data/processed/'+'cdf.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ristomartin/OneDrive/Dropbox/UniStuff/DPhil/Experimental/python_analysis/uCT/flat_sheet/output/processed_flat.csv'

# Plotting

In [ ]:
#Initially import processed flat sheet membrane data
processed_flat = pd.read_csv(save_loc + 'processed_flat.csv',index_col = 0)

#first create figure for new plot of force/extension
fig, ax = plt.subplots()
                
#Before able to plot need to catagorise data by third variable e.g by pyridine conc

#as all data is in a single column and are only plotting a line graph can separate series using pandas groupby
for key, grp in processed_flat.sort_values(['time_spun']).groupby(['pyridine_conc']):
    
    #set the data in each axis
    x = grp['time_spun']
    y = grp['median_thickness_um']

    ax.plot(x,y, label = key)
    #add precalculated IQR bands for each graph for force/extension line graph
    ax.fill_between(grp['time_spun'], grp['median_thickness_um'] - grp['thickness_IQR_um'],grp['median_thickness_um'] + grp['thickness_IQR_um'], alpha=0.35)

#adding formatting into each graph
#force/extension graph
xlabel = 'Time Spun (Hrs)'
ylabel = 'Median Membrane Thickness ($\mu$m)'
ax.legend()
ax.set(xlabel=xlabel, ylabel= ylabel) #(xlabel=x, ylabel='Fibre Diameter ($\mu$m)')

#save figure out
fig.savefig(save_loc+'flat_thickness.png',bbox_inches='tight', dpi=300)